In [1]:
!pip install shap

     |████████████████████████████████| 564 kB 13.4 MB/s 


In [4]:
from sklearn.calibration import calibration_curve
import sklearn
import shap
import pandas as pd
import numpy as np

/home/joaorulff/anaconda3/envs/calibration/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
X,y = shap.datasets.adult(display=True)


le = sklearn.preprocessing.LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])

X = pd.get_dummies(X, columns=["Workclass", "Marital Status", "Occupation", "Relationship", "Race", "Country"])

scaler = sklearn.preprocessing.StandardScaler()

train_col_sacle = X[['Age','Education-Num','Hours per week']]
train_scaler_col = scaler.fit_transform(train_col_sacle)
train_scaler_col = pd.DataFrame(train_scaler_col,columns=train_col_sacle.columns)
X['Age'] = train_scaler_col['Age']
X['Education-Num'] = train_scaler_col['Education-Num']
X['Hours per week'] = train_scaler_col['Hours per week']

In [7]:
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# Generate predictions
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=42, max_iter=100).fit(X_train, y_train)
y_preds = clf.predict_proba(X_valid)

# OHE labels
ohe = sklearn.preprocessing.OneHotEncoder()

y_valid_labels = ohe.fit_transform(y_valid.reshape(-1,1)).toarray()

In [8]:
X_valid

,Age,Education-Num,Sex,Capital Gain,Capital Loss,Hours per week,Workclass_ ?,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Country_ Portugal,Country_ Puerto-Rico,Country_ Scotland,Country_ South,Country_ Taiwan,Country_ Thailand,Country_ Trinadad&Tobago,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia
14160,-0.849080,-0.031360,0,0.0,0.0,-0.197409,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
27048,0.470546,-0.420060,0,0.0,0.0,-0.035429,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
28868,-0.702455,1.134739,1,0.0,0.0,1.179417,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5667,-0.629143,1.134739,0,0.0,0.0,-0.035429,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7827,-0.702455,-0.031360,1,2202.0,0.0,0.774468,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,2.376673,1.134739,0,2329.0,0.0,-1.979184,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
24534,1.203672,-0.031360,1,0.0,0.0,-0.035429,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
18080,0.617171,1.912138,1,99999.0,0.0,0.612489,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10354,-0.849080,1.134739,0,0.0,0.0,-0.035429,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
y_preds

array([[0.99222576, 0.00777424],
       [0.48922159, 0.51077841],
       [0.31770155, 0.68229845],
       ...,
       [0.        , 1.        ],
       [0.98920214, 0.01079786],
       [0.99883025, 0.00116975]])

In [11]:
y_valid_labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [17]:
def reliability_diagram(y_preds, y_valid_labels, class_index=1, bins=10):
    acc, conf = calibration_curve(y_valid_labels[:,class_index], y_preds[:,class_index], n_bins=bins, strategy="uniform")
    return conf, acc

In [18]:
conf, acc = reliability_diagram(y_preds, y_valid_labels, class_index=1, bins=10)

In [24]:
acc

array([0.02625369, 0.18810289, 0.25346535, 0.32947977, 0.5137931 ,
       0.55462185, 0.72272727, 0.73762376, 0.72972973, 0.76910017])